In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
import sys
sys.path.append('..')


import autorobot as ar

for i, t in enumerate(ar.RAnalysisType.items()):
    print(i, t)
    
print(len(list(ar.RAnalysisType)))

print(len(set(int(el) for el in ar.RAnalysisType.values())))

0 ('I_CAT_COMB', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B786C0>)
1 ('I_CAT_STATIC_LINEAR', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78700>)
2 ('I_CAT_STATIC_NONLINEAR', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0FE94AA80>)
3 ('I_CAT_STATIC_INCREMENTAL', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0FE432200>)
4 ('I_CAT_STATIC_BUCKLING', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78740>)
5 ('I_CAT_STATIC_FLAMBEMENT', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78780>)
6 ('I_CAT_STATIC_LINEAR_AUXILIARY', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B787C0>)
7 ('I_CAT_STATIC_NONLINEAR_INCREMENTAL', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78800>)
8 ('I_CAT_STATIC_NONLINEAR_FLAMBEMENT', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78840>)
9 ('I_CAT_STATIC_NONLINEAR_BUCKLING', <RobotOM.IRobotCaseAnalizeType object at 0x000001E0B0B78880>)
10 ('I_CAT_STATIC_NONLINEAR_MODAL', <RobotOM.IRobotCaseAnalizeTy

In [4]:
rb = ar.initialize()
rb.new('SHELL')

from numpy.random import random
from RobotOM import IRobotViewMngr

n1 = rb.nodes.create(*random((3,)))
n2 = rb.nodes.create(*random((3,)))
bar = rb.bars.create(n1, n2)
case = rb.cases.create_case(1, 'case 1', 'PERM', 'LINEAR')

case.add_self_weight(desc='sw')
case.add_bar_udl(bar.Number, fx=random(), desc='udl')
case.add_bar_pl(bar.Number, x=.5, fx=1, is_relative=True, desc='pl')

print(case.Records.Count)
print(dir(case))
print(case.loads)

3
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_inst', '_otype', 'add_bar_pl', 'add_bar_udl', 'add_self_weight', 'delete', 'get', 'get_record_value', 'load_type_values', 'loads', 'set_record_value']
[{'Type': <RobotOM.IRobotLoadRecordType object at 0x0000023CF78B3500>, 'Description': 'sw', 'X': 0.0, 'Y': 0.0, 'Z': -1.0, 'COEFF': 1.0, 'ENTIRE_STRUCT': 1.0}, {'Type': <RobotOM.IRobotLoadRecordType object at 0x0000023CE13BC480>, 'Description': 'udl', 'FX': 732.8444518459831, 'FY': 0.0, 'FZ': 0.0, 'ALPHA': 0.0, 'BETA': 0.0, 'GAMMA': 0.0, 'IS_LOC': 0.0, 'IS_PROJ': 0.0, 'IS_REL': 0.0, 'OFFSET_Y': 0.0, 'OFFSET_Z': 0.0}, {'Type': <Robot

In [5]:
case = rb.cases.get(1)
print(dir(case))
print(case.loads)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_inst', '_otype', 'add_bar_pl', 'add_bar_udl', 'add_self_weight', 'delete', 'get', 'get_record_value', 'load_type_values', 'loads', 'set_record_value']
[{'Type': <RobotOM.IRobotLoadRecordType object at 0x0000023CF775CAC0>, 'Description': 'sw', 'X': 0.0, 'Y': 0.0, 'Z': -1.0, 'COEFF': 1.0, 'ENTIRE_STRUCT': 1.0}, {'Type': <RobotOM.IRobotLoadRecordType object at 0x0000023CF7AF6180>, 'Description': 'udl', 'FX': 732.8444518459831, 'FY': 0.0, 'FZ': 0.0, 'ALPHA': 0.0, 'BETA': 0.0, 'GAMMA': 0.0, 'IS_LOC': 0.0, 'IS_PROJ': 0.0, 'IS_REL': 0.0, 'OFFSET_Y': 0.0, 'OFFSET_Z': 0.0}, {'Type': <RobotOM

In [6]:
rb.new('SHELL')

In [7]:
n1 = rb.nodes.create(0,0,0)
n2 = rb.nodes.create(10,0,0)

In [8]:
bar = rb.bars.create(1, 2)

In [9]:
print(bar.HasLabel(ar.constants.RLabelType.BAR_SECT))
label = bar.section
print(label)

0
None


In [10]:
print(rb.sections.db_list())

rb.sections.load('UC 305x305x97')

bar.section = 'UC 305x305x97'
bar.material = 'S355'

['AISC', 'AISI', 'ARBU', 'ARCLR', 'AUS', 'BSK', 'BSK06', 'BORGA', 'CISC', 'CISC9', 'CHINA', 'CHIN9', 'CORUS', 'DIN', 'DREW', 'ESR', 'EURO', 'EC5', 'GOST', 'GOSTL', 'GOSTT', 'GSTTL', 'INDIA', 'IS808', 'IS127', 'JAPAN', 'KERTO', 'KOREA', 'LIN', 'NA', 'NDS', 'NS34', 'NZL', 'OTUA', 'PRU', 'RARB', 'RCAT', 'RESIN', 'ROMAN', 'RPLN', 'RPLN7', 'RPOL', 'RTRKK', 'SADEF', 'SAISC', 'SIMPL', 'SIN', 'SSMA', 'STAL', 'STO', 'STOL', 'SUOMI', 'THAI', 'TSNL', 'TU', 'UKST', 'UNI', 'WOOD', 'WOODM', 'ZA', 'RUSER', 'SJI', 'VLCFT']


In [11]:
from autorobot.constants import RLoadType
from RobotOM import IRobotLoadRecordType


RLoadType.update({'I_LRT_BODY_FORCE': 89})

rb.cases.create_case(1, 'Body force', 'PERM', 'LINEAR')

In [12]:
from RobotOM import IRobotLoadRecord



In [13]:
case = rb.cases.get(1)
rec = IRobotLoadRecord(case.Records.Get(1))

print(dir(rec))
for i in range(16):
    print(rec.GetValue(i))
print(rec.Type)
rec.Description = 'Body Force'

COMException:  (0x80004005): Error HRESULT E_FAIL has been returned from a call to a COM component.
   at RobotOM.RobotLoadRecordMngrClass.Get(Int32 record_index)

In [ ]:
obj_server = rb.Project.Structure.Objects

In [ ]:
dir(n1.node)

In [ ]:
n1.X = 5

In [ ]:
dir(n1)

In [ ]:
n1.Y = 3.

In [ ]:
n1.Number